In [1]:
import numpy as np
import pandas as pd

## 一、索引器

### 1. 表的列索引
列索引是最常见的索引形式，一般通过 [] 来实现。通过 [列名] 可以从 DataFrame 中取出相应的列，返回值为 Series ，例如从表中取出姓名一列：

In [2]:
df = pd.read_csv('./data/learn_pandas.csv')

df['Name'].head()

0      Gaopeng Yang
1    Changqiang You
2           Mei Sun
3      Xiaojuan Sun
4       Gaojuan You
Name: Name, dtype: object

如果要取出多个列，则可以通过 [列名组成的列表] ，其返回值为一个 DataFrame ，例如从表中取出性别和姓名两列：

In [3]:
df[['Gender','Name']].head()

,Gender,Name
0,Female,Gaopeng Yang
1,Male,Changqiang You
2,Male,Mei Sun
3,Female,Xiaojuan Sun
4,Male,Gaojuan You


此外，若要取出单列，且列名中不包含空格，则可以用 .列名 取出，这和 [列名] 是等价的：

In [4]:
df.Name.head()

0      Gaopeng Yang
1    Changqiang You
2           Mei Sun
3      Xiaojuan Sun
4       Gaojuan You
Name: Name, dtype: object

### 2.序列的行索引

【a】以字符串为索引的Series
如果取出单个索引的对应元素，则可以使用[item],若Series只有单个值对应，则返回这个标量值，如果有多个值对应，则返回一个Series:

In [5]:
s = pd.Series([1,2,3,4,5,6],index=['a','b','a','a','a','c'])

s['a']
# a    1
# a    3
# a    4
# a    5
# dtype: int64

s['b']
# np.int64(2)

np.int64(2)

如果取出多个索引的对应元素，则可以使用 [items的列表] ：

In [6]:
s[['c','b']]

c    6
b    2
dtype: int64

如果想要取出某两个索引之间的元素，并且这两个索引是在整个索引中唯一出现，则可以使用切片，同时需要注意**这里的切片会包含两个端点**：

In [7]:
s['c':'b':-2]
# 包含首尾
# c    6
# a    4
# b    2
# dtype: int64

c    6
a    4
b    2
dtype: int64

如果前后端点的值存在重复，即非唯一值，那么需要经过排序才能使用切片：

In [8]:
s.sort_index()['a': 'b']

a    1
a    3
a    4
a    5
b    2
dtype: int64

【b】以整数为索引的Series

在使用数据的读入函数时，如果不特别指定所对应的列作为索引，那么会生成从0开始的整数索引作为默认索引。当然，任意一组符合长度要求的整数都可以作为索引。

和字符串一样，如果使用 [int] 或 [int_list] ，则可以取出对应索引 元素 的值：

In [9]:
s = pd.Series(['a','b','c','d','e','f'],index=[1,3,1,2,5,4])
s[1]
# 1    a
# 1    c
# dtype: object

s[[2,3]]
# 2    d
# 3    b
# dtype: object

2    d
3    b
dtype: object

如果使用整数切片，则会取出对应索引 位置 的值，注意这里的整数切片同 Python 中的切片一样不包含右端点：

In [10]:
s[1:-1:2]
# 这个索引是从0到n-1，和Series的index不一样。

3    b
2    d
dtype: object

### 3. loc索引器

前面讲到了对 DataFrame 的列进行选取，下面要讨论其行的选取。对于表而言，有两种索引器，一种是基于 元素 的 loc 索引器，另一种是基于 位置 的 iloc 索引器。

loc 索引器的一般形式是 loc[\*, \*] ，**其中第一个 \* 代表行的选择，第二个 * 代表列的选择**，如果省略第二个位置写作 loc[*] ，这个 * 是指行的筛选。其中， * 的位置一共有**五类合法对象**，分别是：**单个元素、元素列表、元素切片、布尔列表以及函数，下面将依次说明**。

为了演示相应操作，先利用 set_index 方法把 Name 列设为索引，关于该函数的其他用法将在多级索引一章介绍。

In [11]:
df_demo = df.set_index('Name')

df_demo

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Gaopeng Yang,A,Freshman,Female,158.9,46.0,N,1,2019/10/5,0:04:34
Changqiang You,B,Freshman,Male,166.5,70.0,N,1,2019/9/4,0:04:20
Mei Sun,A,Senior,Male,188.9,89.0,N,2,2019/9/12,0:05:22
Xiaojuan Sun,C,Sophomore,Female,NaN,41.0,N,2,2020/1/3,0:04:08
Gaojuan You,C,Sophomore,Male,174.0,74.0,N,2,2019/11/6,0:05:22
...,...,...,...,...,...,...,...,...,...
Xiaojuan Sun,C,Junior,Female,153.9,46.0,N,2,2019/10/17,0:04:31
Li Zhao,D,Senior,Female,160.9,50.0,N,3,2019/9/22,0:04:03
Chengqiang Chu,A,Senior,Female,153.9,45.0,N,1,2020/1/5,0:04:48


【a】 * 为单个元素

此时，直接取出相应的行或列，如果该元素在索引中重复则结果为 DataFrame，否则为 Series ：

In [12]:
df_demo.loc['Qiang Sun']

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Qiang Sun,D,Junior,Female,163.1,53.0,N,1,2019/12/11,0:05:08
Qiang Sun,D,Sophomore,Female,154.3,40.0,N,1,2019/12/30,0:04:37
Qiang Sun,A,Junior,Female,160.8,NaN,N,1,2019/9/7,0:04:31


In [13]:
df_demo.loc['Quan Zhao'] # 名字唯一

School                 A
Grade             Junior
Gender            Female
Height             160.6
Weight              53.0
Transfer               N
Test_Number            2
Test_Date      2019/10/4
Time_Record      0:03:45
Name: Quan Zhao, dtype: object

同时选择行和列

In [14]:
df_demo.loc['Qiang Sun','School'] # 返回Series
# Name
# Qiang Sun    D
# Qiang Sun    D
# Qiang Sun    A
# Name: School, dtype: object

df_demo.loc['Quan Zhao','School']
# 'A'

'A'

【b】 * 为元素列表

此时，取出列表中所有元素值对应的行或列：

In [15]:
df_demo.loc[['Qiang Sun','Quan Zhao'],['School','Gender']]

,School,Gender
Name,,
Qiang Sun,D,Female
Qiang Sun,D,Female
Qiang Sun,A,Female
Quan Zhao,A,Female


【c】 * 为切片

之前的 Series 使用字符串索引时提到，**如果是唯一值的起点和终点字符，那么就可以使用切片，并且包含两个端点**，如果不唯一则报错：

In [16]:
df_demo.loc['Gaojuan You':'Gaoqiang Qian','School':'Gender']

,School,Grade,Gender
Name,,,
Gaojuan You,C,Sophomore,Male
Xiaoli Qian,D,Freshman,Female
Qiang Chu,A,Freshman,Female
Gaoqiang Qian,D,Junior,Female


需要注意的是，如果 DataFrame 使用整数索引，其使用整数切片的时候和上面字符串索引的要求一致，都是 元素 切片，包含端点且起点、终点不允许有重复值。

In [17]:
df_loc_slice_demo = df_demo.copy()
# df_demo.shape[0] 200

# start:200,end:1,倒序
df_loc_slice_demo.index = range(df_demo.shape[0],0,-1)

# 没写列，默认所有列
df_loc_slice_demo.loc[5:3]

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
5,C,Junior,Female,153.9,46.0,N,2,2019/10/17,0:04:31
4,D,Senior,Female,160.9,50.0,N,3,2019/9/22,0:04:03
3,A,Senior,Female,153.9,45.0,N,1,2020/1/5,0:04:48


In [18]:
df_loc_slice_demo.loc[3:5] # # 没有返回，说明不是整数位置切片

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record


【d】 * 为布尔列表

在实际的数据处理中，根据条件来筛选行是极其常见的，此处传入 loc 的布尔列表与 DataFrame 长度相同，且列表为 True 的位置所对应的行会被选中， False 则会被剔除。

例如，选出体重超过70kg的学生：

In [19]:
df_demo.loc[df_demo['Weight'] > 70].head()

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Mei Sun,A,Senior,Male,188.9,89.0,N,2,2019/9/12,0:05:22
Gaojuan You,C,Sophomore,Male,174.0,74.0,N,2,2019/11/6,0:05:22
Xiaopeng Zhou,A,Freshman,Male,174.1,74.0,N,1,2019/9/29,0:05:16
Xiaofeng Sun,D,Senior,Male,170.3,71.0,N,2,2019/11/4,0:03:32
Qiang Zheng,A,Senior,Male,183.9,87.0,N,1,2019/12/5,0:04:59


前面所提到的传入元素列表，也可以**通过 isin 方法返回的布尔列表等价写出**，例如选出所有大一和大四的同学信息：

In [20]:
# 选择Grade是Freshman和Senior的行
df_demo.loc[df_demo['Grade'].isin(['Freshman','Senior'])].head()

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Gaopeng Yang,A,Freshman,Female,158.9,46.0,N,1,2019/10/5,0:04:34
Changqiang You,B,Freshman,Male,166.5,70.0,N,1,2019/9/4,0:04:20
Mei Sun,A,Senior,Male,188.9,89.0,N,2,2019/9/12,0:05:22
Xiaoli Qian,D,Freshman,Female,158.0,51.0,N,1,2019/10/31,0:03:47
Qiang Chu,A,Freshman,Female,162.5,52.0,N,1,2019/12/12,0:03:53


对于**复合条件而言，可以用 |（或）, &（且）, ~（取反） 的组合**来实现，**多个条件写到一行需要小括号，否则报错**。例如选出复旦大学中体重超过70kg的大四学生，或者北大男生中体重超过80kg的非大四的学生：

In [21]:
# cond_1 = (df_demo['School'] == 'C') & (df_demo['Grade'] == 'Senior' ) & (df_demo['Weight'] >70)
cond_1 = (df_demo['School'] == 'C') & (df_demo['Grade'] == 'Senior' ) & (df_demo['Weight'] >70)

# cond_2 = (df_demo['School'] == 'B') & ~(df_demo['Grade'] == 'Senior') & (df_demo['Weight'] > 80)
cond_2 = (df_demo['School'] == 'B') & ~(df_demo['Grade'] == 'Senior') & (df_demo['Weight'] > 80)

df_demo.loc[cond_1 | cond_2]

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Qiang Han,B,Freshman,Male,185.3,87.0,N,3,2020/1/7,0:03:58
Chengpeng Zhou,C,Senior,Male,177.1,81.0,N,1,2019/9/5,0:03:38
Changpeng Zhao,B,Freshman,Male,181.3,83.0,N,2,2019/10/24,0:04:08
Chengpeng Qian,C,Senior,Male,177.2,73.0,Y,1,2019/12/19,0:05:18


#### 练习：
select_dtypes 是一个实用函数，它能够从表中选出相应类型的列，若要选出所有数值型的列，只需使用 .select_dtypes('number') ，请利用布尔列表选择的方法结合 DataFrame 的 dtypes 属性在 learn_pandas 数据集上实现这个功能。

有include和exclude两个参数。

In [22]:
# 没有一个是
df.select_dtypes(include='bool')

# df.select_dtypes(include='number')



""
0
1
2
3
4
...
195
196
197
198


【e】 * 为函数

这里的函数，必须以前面的四种合法形式之一为返回值，并且**函数的输入值为 DataFrame 本身**。假设仍然是上述复合条件筛选的例子，可以把逻辑写入一个函数中再返回，需要注意的是函数的形式参数 x 本质上即为 df_demo ：

In [23]:
def condition(df_demo):
    cond_1 = (df_demo['School'] == 'C') & (df_demo['Grade'] == 'Senior' ) & (df_demo['Weight'] >70)

    cond_2 = (df_demo['School'] == 'B') & ~(df_demo['Grade'] == 'Senior') & (df_demo['Weight'] > 80)

    return cond_1 | cond_2

df_demo.loc[condition]

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Qiang Han,B,Freshman,Male,185.3,87.0,N,3,2020/1/7,0:03:58
Chengpeng Zhou,C,Senior,Male,177.1,81.0,N,1,2019/9/5,0:03:38
Changpeng Zhao,B,Freshman,Male,181.3,83.0,N,2,2019/10/24,0:04:08
Chengpeng Qian,C,Senior,Male,177.2,73.0,Y,1,2019/12/19,0:05:18


此外，还支持使用 lambda 表达式，其返回值也同样必须是先前提到的四种形式之一：

In [24]:
df_demo.loc[lambda x:'Quan Zhao',lambda c:'Gender']
# 'Female'

'Female'

由于函数无法返回如 start: end: step 的切片形式，故**返回切片时要用 slice 对象进行包装**：

In [25]:
df_demo.loc[lambda x: slice('Gaojuan You','Gaoqiang Qian')]

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Gaojuan You,C,Sophomore,Male,174.0,74.0,N,2,2019/11/6,0:05:22
Xiaoli Qian,D,Freshman,Female,158.0,51.0,N,1,2019/10/31,0:03:47
Qiang Chu,A,Freshman,Female,162.5,52.0,N,1,2019/12/12,0:03:53
Gaoqiang Qian,D,Junior,Female,161.9,50.0,N,1,2019/9/3,0:03:45


最后需要指出的是，对于 Series 也可以使用 loc 索引，其遵循的原则与 DataFrame 中用于行筛选的 loc[*] 完全一致，此处不再赘述。

#### 不要使用链式赋值
在对表或者序列赋值时，应当在使用一层索引器后直接进行赋值操作，这样做是**由于进行多次索引后赋值是赋在临时返回的 copy 副本上的，而没有真正修改元素**从而报出 SettingWithCopyWarning 警告。例如，下面给出的例子：

In [26]:
df_chain = pd.DataFrame([[0,0],[1,0],[-1,0]], columns=list('AB'))

df_chain
# A	B
# 0	0	0
# 1	1	0
# 2	-1	0


,A,B
0,0,0
1,1,0
2,-1,0


In [27]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings('error')
    try:
        # # 使用方括号列索引后，再使用点的列索引
        df_chain[df_chain.A!=0].B = 1 
    except Warning as w:
        Warning_Msg = w
# print(Warning_Msg)
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead

# df_chain

df_chain.loc[df_chain.A!=0,'B'] = 1

df_chain

,A,B
0,0,0
1,1,1
2,-1,1


### 4. iloc索引器
iloc 的使用与 loc 完全类似，只不过是针对位置进行筛选，在相应的 * 位置处一共也有五类合法对象，分别是：整数、整数列表、整数切片、布尔列表以及函数，函数的返回值必须是前面的四类合法对象中的一个，其输入同样也为 DataFrame 本身。

In [28]:
# (1) 整数
# 第二行第二列
df_demo.iloc[1,1]

# (2) 整数列表
# 前两列两行
df_demo.iloc[[0,1],[0,1]]

# (3) 整数切片
# 切片不包含结束端点
df_demo.iloc[1:4,2:4]

# (4) 函数
# 返回索引为1-4(不包括结束端点)
df_demo.iloc[lambda x: slice(1,4)]

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Changqiang You,B,Freshman,Male,166.5,70.0,N,1,2019/9/4,0:04:20
Mei Sun,A,Senior,Male,188.9,89.0,N,2,2019/9/12,0:05:22
Xiaojuan Sun,C,Sophomore,Female,NaN,41.0,N,2,2020/1/3,0:04:08


在使用布尔列表的时候要特别注意，不能传入 Series 而必须传入序列的 values ，否则会报错。因此，**在使用布尔筛选的时候还是应当优先考虑 loc 的方式。**

例如，选出体重超过80kg的学生：

In [29]:
df_demo.iloc[(df_demo['Weight']>80).values].head()

,School,Grade,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
Name,,,,,,,,,
Mei Sun,A,Senior,Male,188.9,89.0,N,2,2019/9/12,0:05:22
Qiang Zheng,A,Senior,Male,183.9,87.0,N,1,2019/12/5,0:04:59
Qiang Han,B,Freshman,Male,185.3,87.0,N,3,2020/1/7,0:03:58
Chengpeng Zhou,C,Senior,Male,177.1,81.0,N,1,2019/9/5,0:03:38
Feng Han,A,Sophomore,Male,183.4,82.0,N,2,2019/10/25,0:05:10


对 Series 而言同样也可以通过 iloc 返回相应位置的值或子序列：

In [30]:
df_demo['School'].iloc[1]
df_demo['School'].iloc[1:5:2]

Name
Changqiang You    B
Xiaojuan Sun      C
Name: School, dtype: object

### 5. query方法
在 pandas 中，**支持把字符串形式的查询表达式传入 query 方法来查询数据，其表达式的执行结果必须返回布尔列表**。在进行复杂索引时，**由于这种检索方式无需像普通方法一样重复使用 DataFrame 的名字来引用列名**，一般而言会使代码长度在不降低可读性的前提下有所减少。

例如，将 loc 一节中的复合条件查询例子可以如下改写：

In [31]:
cond_1 = (df_demo['School'] == 'C') & (df_demo['Grade'] == 'Senior' ) & (df_demo['Weight'] >70)

cond_2 = (df_demo['School'] == 'B') & ~(df_demo['Grade'] == 'Senior') & (df_demo['Weight'] > 80)


df.query("((School == 'C') & (Grade == 'Senior') & (Weight > 70)) | ((School == 'B') & ~(Grade == 'Senior') & (Weight > 80))")

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
38,B,Freshman,Qiang Han,Male,185.3,87.0,N,3,2020/1/7,0:03:58
66,C,Senior,Chengpeng Zhou,Male,177.1,81.0,N,1,2019/9/5,0:03:38
99,B,Freshman,Changpeng Zhao,Male,181.3,83.0,N,2,2019/10/24,0:04:08
131,C,Senior,Chengpeng Qian,Male,177.2,73.0,Y,1,2019/12/19,0:05:18


在 query 表达式中，**帮用户注册了所有来自 DataFrame 的列名，所有属于该 Series 的方法都可以被调用，和正常的函数调用并没有区别**，例如查询体重超过均值的学生：

In [32]:
df.query('Weight > Weight.mean()').head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
1,B,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,A,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
4,C,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22
10,A,Freshman,Xiaopeng Zhou,Male,174.1,74.0,N,1,2019/9/29,0:05:16
14,D,Senior,Xiaomei Zhou,Female,165.3,57.0,N,1,2019/12/29,0:05:25


- 对于**含有空格的列名**，需要使用 `col name` 的方式进行引用。
- 同时，在 query 中还注册了若干英语的字面用法，帮助提高可读性，例如：**or, and, or, in, not in**。例如，筛选出男生中不是大一大二的学生：

In [33]:
df.query('(Grade not in ["Freshman","Sophomore"]) and '
         '(Gender == "Male")').head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
2,A,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
16,D,Junior,Xiaoqiang Qin,Male,170.1,68.0,N,1,2019/9/11,0:04:51
17,D,Junior,Peng Wang,Male,162.8,65.0,N,1,2019/11/2,0:04:53
18,D,Senior,Xiaofeng Sun,Male,170.3,71.0,N,2,2019/11/4,0:03:32
21,A,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN,1,2020/1/2,0:04:54


此外，在字符串中出现与列表的比较时，**== 和 != 分别表示元素出现在列表和没有出现在列表，等价于 in 和 not in**，例如查询所有大三和大四的学生：

In [34]:
df.query('Grade == ["Junior","Senior"]').head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
2,A,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
7,D,Junior,Gaoqiang Qian,Female,161.9,50.0,N,1,2019/9/3,0:03:45
9,B,Junior,Juan Xu,Female,164.8,NaN,N,3,2019/10/5,0:04:05
11,D,Junior,Xiaoquan Lv,Female,153.2,43.0,N,2,2019/9/16,0:04:49
12,A,Senior,Peng You,Female,NaN,48.0,NaN,2,2019/10/20,0:04:10


对于 **query 中的字符串，如果要引用外部变量，只需在变量名前加 @ 符号**。例如，取出体重位于70kg到80kg之间的学生：

In [35]:
low, high = 70,80
df.query('(Weight >= @low ) & (Weight <= @high)').head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
1,B,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
4,C,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22
10,A,Freshman,Xiaopeng Zhou,Male,174.1,74.0,N,1,2019/9/29,0:05:16
18,D,Senior,Xiaofeng Sun,Male,170.3,71.0,N,2,2019/11/4,0:03:32
35,B,Freshman,Gaoli Zhao,Male,175.4,78.0,N,2,2019/10/8,0:03:32


### 6. 随机抽样

如果把 DataFrame 的每一行看作一个样本，或把每一列看作一个特征，再把整个 DataFrame 看作总体，**想要对样本或特征进行随机抽样就可以用 sample 函数**。有时在拿到大型数据集后，想要对统计特征进行计算来了解数据的大致分布，但是这很费时间。同时，**由于许多统计特征在等概率不放回的简单随机抽样条件下，是总体统计特征的无偏估计，比如样本均值和总体均值**，那么就可以先从整张表中抽出一部分来做近似估计。

sample 函数中的主要参数为 n, axis, frac, replace, weights ，前三个分别是指抽样数量、抽样的方向（0为行、1为列）和抽样比例（0.3则为从总体中抽出30%的样本）。

replace 和 weights 分别是指**是否放回和每个样本的抽样相对概率**，当 replace = True 则表示**有放回抽样**。例如，对下面构造的 df_sample 以 value 值的相对大小为抽样概率进行有放回抽样，抽样数量为3。

In [36]:
df_sample = pd.DataFrame({'id':list('abcde'),'value':[1,2,3,4,500]})

# 样本数量n和frac不能同时出现
df_sample.sample(n=2,replace= True,weights=df_sample.value)
df_sample.sample(frac=0.5,replace= True,weights=df_sample.value)



,id,value
4,e,500
4,e,500


## 二、多级索引

### 1. 多级索引及其表的结构
为了更加清晰地说明具有多级索引的 DataFrame 结构，下面新构造一张表，读者可以忽略这里的构造方法，它们将会在第4小节被更详细地讲解。

In [37]:
np.random.seed(0)
muti_index = pd.MultiIndex.from_product([list('ABCD'),df['Gender'].unique()],names=['School','Gender'])
# MultiIndex([('A', 'Female'),
#             ('A',   'Male'),
#             ('B', 'Female'),
#             ('B',   'Male'),
#             ('C', 'Female'),
#             ('C',   'Male'),
#             ('D', 'Female'),
#             ('D',   'Male')],
#            names=['School', 'Gender'])

muti_column = pd.MultiIndex.from_product([['Height','Weight'],df['Grade'].unique()],names=('indicator','Grade'))

df_multi = pd.DataFrame(np.c_[(np.random.randn(8,4)*5 + 163).tolist(),
                              (np.random.randn(8,4)*5+65).tolist()],
                              index=muti_index,
                              columns=muti_column).round(1) # 保留一位小数
df_multi


indicator       Height                           Weight                   \
Grade         Freshman Senior Sophomore Junior Freshman Senior Sophomore   
School Gender                                                              
A      Female    171.8  165.0     167.9  174.2     60.6   55.1      63.3   
       Male      172.3  158.1     167.8  162.2     71.2   71.0      63.1   
B      Female    162.5  165.1     163.7  170.3     59.8   57.9      56.5   
       Male      166.8  163.6     165.2  164.7     62.5   62.8      58.7   
C      Female    170.5  162.0     164.6  158.7     56.9   63.9      60.5   
       Male      150.2  166.3     167.3  159.3     62.4   59.1      64.9   
D      Female    174.3  155.7     163.2  162.1     65.3   66.5      61.8   
       Male      170.7  170.3     163.8  164.9     61.6   63.2      60.9   

indicator             
Grade         Junior  
School Gender         
A      Female   65.8  
       Male     63.5  
B      Female   74.8  
       Male     68.9  
C      Female   66.9  
       Male     67.1  
D      Female   63.2  
       Male     56.4

In [38]:
df_multi.mean(axis=0)

indicator  Grade    
Height     Freshman     167.3875
           Senior       163.2625
           Sophomore    165.4375
           Junior       164.5500
Weight     Freshman      62.5375
           Senior        62.4375
           Sophomore     61.2125
           Junior        65.8250
dtype: float64

下图通过颜色区分，标记了 DataFrame 的结构。与单层索引的表一样，具备元素值、行索引和列索引三个部分。其中，**这里的行索引和列索引都是 MultiIndex 类型，只不过 索引中的一个元素是元组 而不是单层索引中的标量。例如，行索引的第四个元素为 ("B", "Male") ，列索引的第二个元素为 ("Height", "Senior")**，这里需要注意，外层连续出现相同的值时，第一次之后出现的会被隐藏显示，使结果的可读性增强。

![alt text](https://inter.joyfulpandas.datawhale.club/_images/multi_index.png)

与单层索引类似， MultiIndex 也具有名字属性，图中的 School 和 Gender 分别对应了表的第一层和第二层行索引的名字， Indicator 和 Grade 分别对应了第一层和第二层列索引的名字。

索引的名字和值属性分别可以通过 names 和 values 获得：

In [45]:
# 行索引的名称
df_multi.index.names
# FrozenList(['School', 'Gender'])

# 列索引的名称
df_multi.columns.names
# FrozenList(['indicator', 'Grade'])

# 行索引的值
df_multi.index.values
# array([('A', 'Female'), ('A', 'Male'), ('B', 'Female'), ('B', 'Male'),
    #    ('C', 'Female'), ('C', 'Male'), ('D', 'Female'), ('D', 'Male')],
    #   dtype=object)
# 列索引的值
df_multi.columns.values
# array([('Height', 'Freshman'), ('Height', 'Senior'),
    #    ('Height', 'Sophomore'), ('Height', 'Junior'),
    #    ('Weight', 'Freshman'), ('Weight', 'Senior'),
    #    ('Weight', 'Sophomore'), ('Weight', 'Junior')], dtype=object)

array([('Height', 'Freshman'), ('Height', 'Senior'),
       ('Height', 'Sophomore'), ('Height', 'Junior'),
       ('Weight', 'Freshman'), ('Weight', 'Senior'),
       ('Weight', 'Sophomore'), ('Weight', 'Junior')], dtype=object)

得到某一层的索引，需要通过get_level_values(index)获得：

In [49]:
df_multi.index.get_level_values(0)
# Index(['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D'], dtype='object', name='School')
df_multi.index.get_level_values(1)
# Index(['Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male'], dtype='object', name='Gender')

df_multi.columns.get_level_values(0)
# Index(['Height', 'Height', 'Height', 'Height', 'Weight', 'Weight', 'Weight',
    #    'Weight'],
    #   dtype='object', name='indicator')

df_multi.columns.get_level_values(1)
# Index(['Freshman', 'Senior', 'Sophomore', 'Junior', 'Freshman', 'Senior',
    #    'Sophomore', 'Junior'],
    #   dtype='object', name='Grade')

Index(['Freshman', 'Senior', 'Sophomore', 'Junior', 'Freshman', 'Senior',
       'Sophomore', 'Junior'],
      dtype='object', name='Grade')

但**对于索引而言，无论是单层还是多层，用户都无法通过 index_obj[0] = item 的方式来修改元素，也不能通过 index_name[0] = new_name 的方式来修改名字**，关于如何修改这些属性的话题将在第三节被讨论

### 2. 多级索引中的loc索引器
熟悉了结构后，现在回到原表，将学校和年级设为索引，此时的行为多级索引，列为单级索引，**由于默认状态的列索引不含名字，因此对应于刚刚图中 Indicator 和 Grade 的索引名位置是空缺的**。

In [50]:
df_multi = df.set_index(['School','Grade'])
df_multi

Name  Gender  Height  Weight Transfer  \
School Grade                                                        
A      Freshman     Gaopeng Yang  Female   158.9    46.0        N   
B      Freshman   Changqiang You    Male   166.5    70.0        N   
A      Senior            Mei Sun    Male   188.9    89.0        N   
C      Sophomore    Xiaojuan Sun  Female     NaN    41.0        N   
       Sophomore     Gaojuan You    Male   174.0    74.0        N   
...                          ...     ...     ...     ...      ...   
       Junior       Xiaojuan Sun  Female   153.9    46.0        N   
D      Senior            Li Zhao  Female   160.9    50.0        N   
A      Senior     Chengqiang Chu  Female   153.9    45.0        N   
       Senior      Chengmei Shen    Male   175.3    71.0        N   
D      Sophomore     Chunpeng Lv    Male   155.7    51.0        N   

                  Test_Number   Test_Date Time_Record  
School Grade                                           
A      Freshman             1   2019/10/5     0:04:34  
B      Freshman             1    2019/9/4     0:04:20  
A      Senior               2   2019/9/12     0:05:22  
C      Sophomore            2    2020/1/3     0:04:08  
       Sophomore            2   2019/11/6     0:05:22  
...                       ...         ...         ...  
       Junior               2  2019/10/17     0:04:31  
D      Senior               3   2019/9/22     0:04:03  
A      Senior               1    2020/1/5     0:04:48  
       Senior               2    2020/1/7     0:04:58  
D      Sophomore            1   2019/11/6     0:05:05  

[200 rows x 8 columns]

由于多级索引中的单个元素以元组为单位，因此之前在第一节介绍的 loc 和 iloc 方法完全可以照搬，**只需把标量的位置替换成对应的元组**。

当传入**元组列表或单个元组或返回前二者的函数时，需要先进行索引排序**以避免性能警告：

In [55]:
with warnings.catch_warnings():
    warnings.filterwarnings('error')
    try:
        df_multi.loc[('Fudan University', 'Junior')].head()
    except Warning as w:
        Warning_Msg = w


# Warning_Msg
# pandas.errors.PerformanceWarning('indexing past lexsort depth may impact performance.')

# 排序
df_sorted = df_multi.sort_index()
# 单个元素
df_sorted.loc[('C','Junior')].head()
# 元素列表
df_sorted.loc[[('C','Senior'),('B','Freshman')]].head()
# 布尔列表
df_sorted.loc[df_sorted['Weight'] > 70].head()
# 函数
df_sorted.loc[lambda x:('C','Junior')].head()

Name  Gender  Height  Weight Transfer  Test_Number  \
School Grade                                                                 
C      Junior      Yanli You  Female     NaN    48.0        N            1   
       Junior  Chunqiang Chu    Male   168.6    72.0        N            2   
       Junior   Changfeng Lv    Male   175.6    76.0        N            2   
       Junior     Yanjuan Lv  Female   159.3    49.0      NaN            1   
       Junior  Gaoqiang Zhou  Female   156.8    43.0        N            1   

                Test_Date Time_Record  
School Grade                           
C      Junior   2019/9/23     0:03:34  
       Junior  2019/10/16     0:03:58  
       Junior   2019/9/11     0:04:35  
       Junior    2019/9/3     0:03:39  
       Junior   2019/11/4     0:04:34

当使用切片时需要注意，在单级索引中只要切片端点元素是唯一的，那么就可以进行切片，但**在多级索引中，无论元组在索引中是否重复出现，都必须经过排序才能使用切片**，否则报错：

In [66]:
try:
    df_multi.loc[('Fudan University', 'Senior'):].head()
except Exception as e:
    Err_Msg = e


Err_Msg
# pandas.errors.UnsortedIndexError('Key length (2) was greater than MultiIndex lexsort depth (0)')

# 排序后，才能使用切片索引
df_sorted.loc[('C', 'Senior'):].head()

# 去重df的School和Grade列，并设置School和Grade列为索引
df_unique = df.drop_duplicates(subset=['School','Grade']).set_index(['School','Grade'])

df_unique.head()

# 排序index后再进行切片索引
df_unique.sort_index().loc[('C','Senior'):].head()


Name  Gender  Height  Weight Transfer  \
School Grade                                                         
C      Senior     Chengpeng Zheng  Female   151.7    38.0        N   
       Sophomore     Xiaojuan Sun  Female     NaN    41.0        N   
D      Freshman       Xiaoli Qian  Female   158.0    51.0        N   
       Junior       Gaoqiang Qian  Female   161.9    50.0        N   
       Senior        Xiaomei Zhou  Female   165.3    57.0        N   

                  Test_Number   Test_Date Time_Record  
School Grade                                           
C      Senior               2   2019/11/1     0:03:39  
       Sophomore            2    2020/1/3     0:04:08  
D      Freshman             1  2019/10/31     0:03:47  
       Junior               1    2019/9/3     0:03:45  
       Senior               1  2019/12/29     0:05:25

此外，在多级索引中的元组有一种特殊的用法，**可以对多层的元素进行交叉组合后索引，但同时需要指定 loc 的列，全选则用 : 表示**。其中，每一层需要选中的元素用列表存放，传入 loc 的形式为 [(level_0_list, level_1_list), cols] 。例如，想要得到**所有北大和复旦的大二大三学生**，可以如下写出：

In [72]:
res = df_multi.loc[(['B','C'],['Sophomore','Junior']),:]
res.head()

res.shape
# (33, 8)

(33, 8)

下面的语句和上面类似，但仍然传入的是元素（这里为元组）的列表，它们的意义是不同的，表示的是**选出北大的大三学生和复旦的大二学生**：

In [75]:
res = df_multi.loc[[('B','Junior'),('C','Sophomore')]]

res.head()

Name  Gender  Height  Weight Transfer  Test_Number  \
School Grade                                                                 
B      Junior        Juan Xu  Female   164.8     NaN        N            3   
       Junior  Changjuan You  Female   161.4    47.0        N            1   
       Junior       Gaoli Xu  Female   157.3    48.0        N            2   
       Junior   Gaoquan Zhou    Male   166.8    70.0        N            2   
       Junior      Qiang You  Female   170.0    56.0        N            3   

                Test_Date Time_Record  
School Grade                           
B      Junior   2019/10/5     0:04:05  
       Junior   2019/10/5     0:04:08  
       Junior  2019/12/11     0:05:13  
       Junior    2019/9/5     0:04:24  
       Junior  2019/12/31     0:04:27

### 3. IndexSlice对象
前面介绍的方法，即使在索引不重复的时候，也**只能对元组整体进行切片，而不能对每层进行切片，也不允许将切片和布尔列表混合使用，引入 IndexSlice 对象就能解决这个问题**。 Slice 对象一共有两种形式，第一种为 loc[idx[*,*]] 型，第二种为 loc[idx[*,*],idx[*,*]] 型，下面将进行介绍。为了方便演示，下面构造一个 索引不重复的 DataFrame ：

In [76]:
np.random.seed(0)

L1,L2 = list('ABC'),list('abc')

muti_index1 = pd.MultiIndex.from_product([L1,L2],names=('Upper','Lower'))

L3,L4 = list('DEF'),list('def')

muti_index2 = pd.MultiIndex.from_product([L3,L4],names=('Big','Small'))

# 组合DataFrame
df_ex = pd.DataFrame(np.random.randint(-9,10,(9,9)),
                     index=muti_index1,columns=muti_index2)
df_ex
# 	Big	D	E	F
# Small	d	e	f	d	e	f	d	e	f
# Upper	Lower									
# A	a	3	6	-9	-6	-6	-2	0	9	-5
# b	-3	3	-8	-3	-2	5	8	-4	4
# c	-1	0	7	-4	6	6	-9	9	-6
# B	a	8	5	-2	-9	-8	0	-9	1	-6
# b	2	9	-7	-9	-9	-5	-4	-3	-1
# c	8	6	-5	0	1	-8	-8	-2	0
# C	a	-6	-3	2	5	9	-9	5	-6	3
# b	1	2	-5	-3	-5	6	-6	3	-5
# c	-1	5	6	-6	6	4	7	8	-4


Big          D        E        F      
Small        d  e  f  d  e  f  d  e  f
Upper Lower                           
A     a      3  6 -9 -6 -6 -2  0  9 -5
      b     -3  3 -8 -3 -2  5  8 -4  4
      c     -1  0  7 -4  6  6 -9  9 -6
B     a      8  5 -2 -9 -8  0 -9  1 -6
      b      2  9 -7 -9 -9 -5 -4 -3 -1
      c      8  6 -5  0  1 -8 -8 -2  0
C     a     -6 -3  2  5  9 -9  5 -6  3
      b      1  2 -5 -3 -5  6 -6  3 -5
      c     -1  5  6 -6  6  4  7  8 -4

为了使用 silce 对象，先要进行定义：

In [77]:
idx = pd.IndexSlice

【a】 loc[idx[\*,\*]] 型

这种情况并不能进行多层分别切片，**前一个 * 表示行的选择，后一个 * 表示列的选择，与单纯的 loc 是类似的**：

In [79]:
# 和普通的loc没有区别
df_ex.loc[idx['C':,('D','f'):]]
df_ex.loc[('C'):,('D','f'):]

Big          D  E        F      
Small        f  d  e  f  d  e  f
Upper Lower                     
C     a      2  5  9 -9  5 -6  3
      b     -5 -3 -5  6 -6  3 -5
      c      6 -6  6  4  7  8 -4

另外，也支持布尔序列的索引：

In [81]:
# 列和大于0的
df_ex.loc[idx[:'A', lambda x:x.sum()>0]] # 列和大于0

df_ex.loc[:'A', lambda x:x.sum()>0]

Big          D     F
Small        d  e  e
Upper Lower         
A     a      3  6  9
      b     -3  3 -4
      c     -1  0  9

【b】 loc[idx[\*,\*],idx[\*,\*]] 型

这种情况能够分层进行切片，前一个 idx 指代的是行索引，后一个是列索引。

不支持函数

In [82]:
# 行：第一层索引到A结束，第二层索引'b'开始
# 列：第一层column E开始，第二层 'e'开始
df_ex.loc[idx[:'A','b':],idx['E':,'e':]]

Big          E     F   
Small        e  f  e  f
Upper Lower            
A     b     -2  5 -4  4
      c      6  6  9 -6

### 4. 多级索引的构造
前面提到了多级索引表的结构和切片，那么除了使用 set_index 之外，如何自己构造多级索引呢？**常用的有 from_tuples, from_arrays, from_product 三种方法**，它们都是 pd.MultiIndex 对象下的函数。

from_tuples 指根据传入由元组组成的列表进行构造：

In [ ]:
# my_tuple = [('a','cat'),('a','dog'),('b','cat'),('b','dog')]
# 通过zip会容易构建一些
my_tuple = zip(list('aabb'),['cat','dog','cat','dog'])

pd.MultiIndex.from_tuples(my_tuple,names=['First','Second'])
# MultiIndex([('a', 'cat'),
#             ('a', 'dog'),
#             ('b', 'cat'),
#             ('b', 'dog')],
#            names=['First', 'Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

from_arrays 指根据传入列表中，对应层的列表进行构造：

In [ ]:
my_array = [list('aabb'),['cat','dog']*2]
pd.MultiIndex.from_arrays(my_array,names=['First','Second'])
# MultiIndex([('a', 'cat'),
#             ('a', 'dog'),
#             ('b', 'cat'),
#             ('b', 'dog')],
#            names=['First', 'Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

from_product 指**根据给定多个列表的笛卡尔积进行构造**

In [ ]:
my_list1,my_list2 = ['a','b'],['cat','dog']
pd.MultiIndex.from_product([my_list1,my_list2],names=['First','Second'])

# MultiIndex([('a', 'cat'),
#             ('a', 'dog'),
#             ('b', 'cat'),
#             ('b', 'dog')],
#            names=['First', 'Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

## 三、索引的常用方法

### 1. 索引层的交换和删除
为了方便理解交换的过程，这里构造一个三级索引的例子：

In [89]:
np.random.seed(0)
L1,L2,L3 = list('AB'),list('ab'),['alpha','beta']

muti_index1 = pd.MultiIndex.from_product([L1,L2,L3],names=('Upper','Lower','Extra'))

L4,L5,L6 = list('CD'),list('cd'),['cat','dog']

muti_index2 = pd.MultiIndex.from_product([L4,L5,L6],names=['Big','Small','Other'])

df_ex = pd.DataFrame(np.random.randint(-9,10,(8,8)),index=muti_index1,columns=muti_index2)
df_ex

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9
            beta   -9  -5  -4  -3  -1   8   6  -5
      b     alpha   0   1  -8  -8  -2   0  -6  -3
            beta    2   5   9  -9   5  -6   3   1

索引层的交换**由 swaplevel 和 reorder_levels 完成，前者只能交换两个层，而后者可以交换任意层**，两者都可以指定交换的是轴是哪一个，即行索引或列索引：

In [90]:
# 列索引的第一层和第三层交换
df_ex.swaplevel(0,2,axis=1).head()

Other             cat dog cat dog cat dog cat dog
Small               c   c   d   d   c   c   d   d
Big                 C   C   C   C   D   D   D   D
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9

In [ ]:
# 行索引交换
df_ex.reorder_levels([2,0,1],axis=0).head()

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Extra Upper Lower                                
alpha A     a       3   6  -9  -6  -6  -2   0   9
beta  A     a      -5  -3   3  -8  -3  -2   5   8
alpha A     b      -4   4  -1   0   7  -4   6   6
beta  A     b      -9   9  -6   8   5  -2  -9  -8
alpha B     a       0  -9   1  -6   2   9  -7  -9

**轴之间的索引交换**

这里**只涉及行或列索引内部的交换**，不同方向索引之间的交换将在第五章中被讨论。

若想要删除某一层的索引，可以使用 droplevel 方法：

In [92]:
df_ex.droplevel(1,axis=1)

Big                 C               D            
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9
            beta   -9  -5  -4  -3  -1   8   6  -5
      b     alpha   0   1  -8  -8  -2   0  -6  -3
            beta    2   5   9  -9   5  -6   3   1

In [93]:
df_ex.droplevel([0,1],axis=0)

Big     C               D            
Small   c       d       c       d    
Other cat dog cat dog cat dog cat dog
Extra                                
alpha   3   6  -9  -6  -6  -2   0   9
beta   -5  -3   3  -8  -3  -2   5   8
alpha  -4   4  -1   0   7  -4   6   6
beta   -9   9  -6   8   5  -2  -9  -8
alpha   0  -9   1  -6   2   9  -7  -9
beta   -9  -5  -4  -3  -1   8   6  -5
alpha   0   1  -8  -8  -2   0  -6  -3
beta    2   5   9  -9   5  -6   3   1

### 2. 索引属性的修改
通过 **rename_axis** 可以对索引层的名字进行修改，常用的修改方式是**传入字典的映射**：

In [94]:
df_ex.rename_axis(index={'Upper':'Changed_row'},
                  columns={'Other':'Changed_col'}).head()

Big                       C               D            
Small                     c       d       c       d    
Changed_col             cat dog cat dog cat dog cat dog
Changed_row Lower Extra                                
A           a     alpha   3   6  -9  -6  -6  -2   0   9
                  beta   -5  -3   3  -8  -3  -2   5   8
            b     alpha  -4   4  -1   0   7  -4   6   6
                  beta   -9   9  -6   8   5  -2  -9  -8
B           a     alpha   0  -9   1  -6   2   9  -7  -9

通过 rename 可以对索引的值进行修改，如果是**多级索引需要指定修改的层号 level** ：

In [95]:
df_ex.rename(columns={'cat':'not_cat'},level=2).head()

Big                     C                       D                
Small                   c           d           c           d    
Other             not_cat dog not_cat dog not_cat dog not_cat dog
Upper Lower Extra                                                
A     a     alpha       3   6      -9  -6      -6  -2       0   9
            beta       -5  -3       3  -8      -3  -2       5   8
      b     alpha      -4   4      -1   0       7  -4       6   6
            beta       -9   9      -6   8       5  -2      -9  -8
B     a     alpha       0  -9       1  -6       2   9      -7  -9

传入参数也可以是函数，其输入值就是索引元素：

In [96]:
df_ex.rename(index=lambda x:str.upper(x),level=2).head()

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     ALPHA   3   6  -9  -6  -6  -2   0   9
            BETA   -5  -3   3  -8  -3  -2   5   8
      b     ALPHA  -4   4  -1   0   7  -4   6   6
            BETA   -9   9  -6   8   5  -2  -9  -8
B     a     ALPHA   0  -9   1  -6   2   9  -7  -9

#### 练习
尝试在 rename_axis 中使用函数完成与例子中一样的功能，即把 Upper 和 Other 分别替换为 Changed_row 和 Changed_col。

In [97]:
df_ex.rename_axis(index=lambda x: 'Changed_row' if x == 'Upper' else x,
                  columns=lambda x: 'Other' if x == 'Changed_col' else x).head()

Big                       C               D            
Small                     c       d       c       d    
Other                   cat dog cat dog cat dog cat dog
Changed_row Lower Extra                                
A           a     alpha   3   6  -9  -6  -6  -2   0   9
                  beta   -5  -3   3  -8  -3  -2   5   8
            b     alpha  -4   4  -1   0   7  -4   6   6
                  beta   -9   9  -6   8   5  -2  -9  -8
B           a     alpha   0  -9   1  -6   2   9  -7  -9

对于整个索引的元素替换，可以利用迭代器实现：

In [ ]:
new_values = iter(list('abcdefgh'))
df_ex.rename(index=lambda x:next(new_values),level=2)


Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     a       3   6  -9  -6  -6  -2   0   9
            b      -5  -3   3  -8  -3  -2   5   8
      b     c      -4   4  -1   0   7  -4   6   6
            d      -9   9  -6   8   5  -2  -9  -8
B     a     e       0  -9   1  -6   2   9  -7  -9
            f      -9  -5  -4  -3  -1   8   6  -5
      b     g       0   1  -8  -8  -2   0  -6  -3
            h       2   5   9  -9   5  -6   3   1

若想要对某个位置的元素进行修改，在单层索引时容易实现，即先取出索引的 values 属性，再给对得到的列表进行修改，最后再对 index 对象重新赋值。但是如果是多级索引的话就有些麻烦，一个解决的方案是先把某一层索引临时转为表的元素，然后再进行修改，最后重新设定为索引，下面一节将介绍这些操作。

另外一个需要介绍的函数是**map ，它是定义在 Index 上的方法，与前面 rename 方法中层的函数式用法是类似的，只不过它传入的不是层的标量值，而是直接传入索引的元组**，这为用户进行跨层的修改提供了便利。例如，可以等价地写出上面的字符串转大写的操作：

In [100]:
df_temp = df_ex.copy()
new_idx = df_temp.index.map(lambda x: (x[0],x[1],x[2].upper()))

new_idx

MultiIndex([('A', 'a', 'ALPHA'),
            ('A', 'a',  'BETA'),
            ('A', 'b', 'ALPHA'),
            ('A', 'b',  'BETA'),
            ('B', 'a', 'ALPHA'),
            ('B', 'a',  'BETA'),
            ('B', 'b', 'ALPHA'),
            ('B', 'b',  'BETA')],
           names=['Upper', 'Lower', 'Extra'])

In [101]:
df_temp.index = new_idx
df_temp.head()

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     ALPHA   3   6  -9  -6  -6  -2   0   9
            BETA   -5  -3   3  -8  -3  -2   5   8
      b     ALPHA  -4   4  -1   0   7  -4   6   6
            BETA   -9   9  -6   8   5  -2  -9  -8
B     a     ALPHA   0  -9   1  -6   2   9  -7  -9

关于 map 的另一个使用方法是**对多级索引的压缩**，这在第四章和第五章的一些操作中是有用的：

In [102]:
df_temp = df_ex.copy()

new_idx = df_temp.index.map(lambda x: (x[0]+'-'+
                                       x[1]+'-'+
                                       x[2]))
df_temp.index = new_idx
df_temp.head() # 单层索引

Big         C               D            
Small       c       d       c       d    
Other     cat dog cat dog cat dog cat dog
A-a-alpha   3   6  -9  -6  -6  -2   0   9
A-a-beta   -5  -3   3  -8  -3  -2   5   8
A-b-alpha  -4   4  -1   0   7  -4   6   6
A-b-beta   -9   9  -6   8   5  -2  -9  -8
B-a-alpha   0  -9   1  -6   2   9  -7  -9

同时，也可以反向地展开：

In [103]:
new_idx = df_temp.index.map(lambda x:tuple(x.split('-')))
df_temp.index = new_idx
df_temp.head() # 三层索引

Big         C               D            
Small       c       d       c       d    
Other     cat dog cat dog cat dog cat dog
A a alpha   3   6  -9  -6  -6  -2   0   9
    beta   -5  -3   3  -8  -3  -2   5   8
  b alpha  -4   4  -1   0   7  -4   6   6
    beta   -9   9  -6   8   5  -2  -9  -8
B a alpha   0  -9   1  -6   2   9  -7  -9

### 3. 索引的设置与重置
为了说明本节的函数，下面构造一个新表：

In [104]:
df_new = pd.DataFrame({'A':list('aacd'),
                       'B':list('PQRT'),
                       'C':[1,2,3,4]})
df_new

,A,B,C
0,a,P,1
1,a,Q,2
2,c,R,3
3,d,T,4


索引的设置可以使用 set_index 完成，这里的**主要参数是 append ，表示是否来保留原来的索引，直接把新设定的添加到原索引的内层**：

In [ ]:
df_new.set_index('A')
# 	B	C
# A		
# a	P	1
# a	Q	2
# c	R	3
# d	T	4

# 直接把新设定的添加到原索引的内层
df_new.set_index('A',append=True)

,,B,C
,A,,
0,a,P,1
1,a,Q,2
2,c,R,3
3,d,T,4


可以同时指定多个列作为索引：

In [ ]:
df_new.set_index(['A', 'B'])

# 
#       C
# A	B	
# a	P	1
# Q	2
# c	R	3
# d	T	4


C
A B   
a P  1
  Q  2
c R  3
d T  4

如果想要添加索引的列没有出现在其中，那么可以直接在参数中传入相应的 Series ：

In [108]:
my_index = pd.Series(list('WXYZ'),name='D')

df_new = df_new.set_index(['A',my_index])
df_new

B  C
A D      
a W  P  1
  X  Q  2
c Y  R  3
d Z  T  4

**reset_index** 是 set_index 的逆函数，其主要参数是 drop ，表示是否**要把去掉的索引层丢弃，而不是添加到列中(default=False)**：

In [ ]:
df_new.reset_index(['D'])
# 	D	B	C
# A			
# a	W	P	1
# a	X	Q	2
# c	Y	R	3
# d	Z	T	4

df_new.reset_index(['D'],drop=True)
# 	B	C
# A		
# a	P	1
# a	Q	2
# c	R	3
# d	T	4


,B,C
A,,
a,P,1
a,Q,2
c,R,3
d,T,4


如果重置了所有的索引，那么 pandas 会直接重新生成一个默认索引：

In [ ]:
df_new.reset_index()

# A	D	B	C
# 0	a	W	P	1
# 1	a	X	Q	2
# 2	c	Y	R	3
# 3	d	Z	T	4


,A,D,B,C
0,a,W,P,1
1,a,X,Q,2
2,c,Y,R,3
3,d,Z,T,4


### 4. 索引的变形
在某些场合下，需要对索引做一些扩充或者剔除，更具体地要求是**给定一个新的索引，把原表中相应的索引对应元素填充到新索引构成的表中**。例如，下面的表中给出了员工信息，需要重新制作一张新的表，要求增加一名员工的同时去掉身高列并增加性别列：

In [ ]:
df_reindex = pd.DataFrame({'Weight':[60,70,80],
                           'Height':[176,180,179]},
                           index=['1001','1003','1002'])
df_reindex
# 	Weight	Height
# 1001	60	176
# 1003	70	180
# 1002	80	179

df_reindex.reindex(index=['1001','1002','1003','1004'],
                   columns=['Weight','Gender'])

# Weight	Gender
# 1001	60.0	NaN
# 1002	80.0	NaN
# 1003	70.0	NaN
# 1004	NaN	NaN


,Weight,Gender
1001,60.0,NaN
1002,80.0,NaN
1003,70.0,NaN
1004,NaN,NaN


这种需求**常出现在时间序列索引的时间点填充以及 ID 编号的扩充**。另外，需要**注意的是原来表中的数据和新表中会根据索引自动对齐**，例如原先的1002号位置在1003号之后，而新表中相反，那么 reindex 中会根据元素对齐，与位置无关。

还有一个与 reindex 功能类似的函数是 **reindex_like ，其功能是仿照传入的表索引来进行被调用表索引的变形**。例如，现在已经存在一张表具备了目标索引的条件，那么上述功能可采用下述代码得到：

In [114]:
df_existed = pd.DataFrame(index=['1001','1002','1003','1004'],
                          columns=['Weight','Gender'])
df_reindex.reindex_like(df_existed)

,Weight,Gender
1001,60.0,NaN
1002,80.0,NaN
1003,70.0,NaN
1004,NaN,NaN


## 四、索引运算
### 1. 集合的运算法则

经常会有一种利用集合运算来取出符合条件行的需求，例如有两张表 A 和 B ，它们的索引都是员工编号，现在需要筛选出两表索引交集的所有员工信息，此时通过 Index 上的运算操作就很容易实现。

不过在此之前，不妨先复习一下常见的四种集合运算：
$$
\begin{align*}
\mathrm{S_A.intersection(S_B)} &= \mathrm{S_A \cap S_B} \Leftrightarrow \{x | x \in \mathrm{S_A} \text{ and } x \in \mathrm{S_B}\} \\
\mathrm{S_A.union(S_B)} &= \mathrm{S_A \cup S_B} \Leftrightarrow \{x | x \in \mathrm{S_A} \text{ or } x \in \mathrm{S_B}\} \\
\mathrm{S_A.difference(S_B)} &= \mathrm{S_A - S_B} \Leftrightarrow \{x | x \in \mathrm{S_A} \text{ and } x \notin \mathrm{S_B}\} \\
\mathrm{S_A.symmetric\_difference(S_B)} &= \mathrm{S_A \triangle S_B} \Leftrightarrow \{x | x \in \mathrm{S_A \cup S_B - S_A \cap S_B}\}
\end{align*}
$$

In [115]:
import pandas as pd

# 1. 定义两个基础集合（用 pd.Index 模拟数学集合）
S_A = pd.Index([1, 2, 3, 4], name="S_A")  # 集合 A: {1,2,3,4}
S_B = pd.Index([3, 4, 5, 6], name="S_B")  # 集合 B: {3,4,5,6}

print("=== 原始集合 ===")
print(f"S_A: {S_A}")
print(f"S_B: {S_B}\n")


# 2. 1. 交集（intersection）：S_A ∩ S_B → {x | x∈S_A 且 x∈S_B}
intersection_result = S_A.intersection(S_B)
# 等价写法：S_A & S_B（注意用括号包裹，避免运算符优先级问题）
# intersection_result = S_A & S_B  
print("=== 1. 交集（intersection）===")
print(f"结果: {intersection_result}")  # 输出: [3,4]
print(f"数学定义验证: {intersection_result} → {{x | x∈S_A 且 x∈S_B}}\n")


# 3. 2. 并集（union）：S_A ∪ S_B → {x | x∈S_A 或 x∈S_B}（去重）
union_result = S_A.union(S_B)
# 等价写法：S_A | S_B
# union_result = S_A | S_B  
print("=== 2. 并集（union）===")
print(f"结果: {union_result}")  # 输出: [1,2,3,4,5,6]
print(f"数学定义验证: {union_result} → {{x | x∈S_A 或 x∈S_B}}\n")


# 4. 3. 差集（difference）：S_A - S_B → {x | x∈S_A 且 x∉S_B}
difference_result = S_A.difference(S_B)
# 等价写法：S_A - S_B（注意 S_B - S_A 是另一种结果）
# difference_result = S_A - S_B  
print("=== 3. 差集（difference）===")
print(f"结果: {difference_result}")  # 输出: [1,2]
print(f"数学定义验证: {difference_result} → {{x | x∈S_A 且 x∉S_B}}\n")


# 5. 4. 对称差集（symmetric_difference）：S_A △ S_B → {x | x∈S_A∪S_B - S_A∩S_B}
sym_diff_result = S_A.symmetric_difference(S_B)
print("=== 4. 对称差集（symmetric_difference）===")
print(f"结果: {sym_diff_result}")  # 输出: [1,2,5,6]
# 数学定义验证：并集减交集
union = S_A.union(S_B)
intersection = S_A.intersection(S_B)
manual_sym_diff = union.difference(intersection)
print(f"数学定义验证: {sym_diff_result} → {manual_sym_diff}（与并集减交集结果一致）\n")

=== 原始集合 ===
S_A: Index([1, 2, 3, 4], dtype='int64', name='S_A')
S_B: Index([3, 4, 5, 6], dtype='int64', name='S_B')

=== 1. 交集（intersection）===
结果: Index([3, 4], dtype='int64')
数学定义验证: Index([3, 4], dtype='int64') → {x | x∈S_A 且 x∈S_B}

=== 2. 并集（union）===
结果: Index([1, 2, 3, 4, 5, 6], dtype='int64')
数学定义验证: Index([1, 2, 3, 4, 5, 6], dtype='int64') → {x | x∈S_A 或 x∈S_B}

=== 3. 差集（difference）===
结果: Index([1, 2], dtype='int64')
数学定义验证: Index([1, 2], dtype='int64') → {x | x∈S_A 且 x∉S_B}

=== 4. 对称差集（symmetric_difference）===
结果: Index([1, 2, 5, 6], dtype='int64')
数学定义验证: Index([1, 2, 5, 6], dtype='int64') → Index([1, 2, 5, 6], dtype='int64')（与并集减交集结果一致）



### 2. 一般的索引运算

由于集合的元素是互异的，但是索引中可能有相同的元素，先用 unique 去重后再进行运算。下面构造两张最为简单的示例表进行演示：

In [ ]:
df_set_1 = pd.DataFrame([[0,1],[1,2],[3,4]],index=pd.Index(['a','b','a'],name= 'id1'))
df_set_2 = pd.DataFrame([[4,5],[2,6],[7,1]],index=pd.Index(['b','b','c'],name= 'id2'))

# 索引元素去重
id1,id2 = df_set_1.index.unique(),df_set_2.index.unique()

id1,id2
# (Index(['a', 'b'], dtype='object', name='id1'),
#  Index(['b', 'c'], dtype='object', name='id2'))

df_set_1.loc[id1]
# 	0	1
# id1		
# a	0	1
# a	3	4
# b	1	2


# 交集
id1.intersection(id2)
# Index(['b'], dtype='object')

# 并集
id1.union(id2)
# Index(['a', 'b', 'c'], dtype='object')

# 差集（difference）
id1.difference(id2)
# Index(['a'], dtype='object')

# 对称差集（symmetric_difference）
id1.symmetric_difference(id2)
# Index(['a', 'c'], dtype='object')

Index(['a', 'c'], dtype='object')

若两张表需要做集合运算的列并没有被设置索引，一种办法是先转成索引，运算后再恢复，另一种方法是利用 isin 函数，例如在重置索引的第一张表中选出id列交集的所在行：

In [126]:
df_set_in_col_1 = df_set_1.reset_index()
df_set_in_col_2 = df_set_2.reset_index()
df_set_in_col_1

# 	id1	0	1
# 0	a	0	1
# 1	b	1	2
# 2	a	3	4

df_set_in_col_2

# id2	0	1
# 0	b	4	5
# 1	b	2	6
# 2	c	7	1

# 交集
df_set_in_col_1[df_set_in_col_1.id1.isin(df_set_in_col_2.id2)]

,id1,0,1
1,b,1,2


## 五、练习
### Ex1：公司员工数据集
现有一份公司员工数据集：
```python
df = pd.read_csv('data/company.csv')

df.head(3)
Out[179]: 
   EmployeeID birthdate_key  age  city_name department      job_title gender
0        1318      1/3/1954   61  Vancouver  Executive            CEO      M
1        1319      1/3/1957   58  Vancouver  Executive      VP Stores      F
2        1320      1/2/1955   60  Vancouver  Executive  Legal Counsel      F
```

1. 分别只使用 query 和 loc 选出年龄不超过四十岁且工作部门为 Dairy 或 Bakery 的男性。

2. 选出员工 ID 号 为奇数所在行的第1、第3和倒数第2列。

3. 按照以下步骤进行索引操作：

- 把后三列设为索引后交换内外两层

- 恢复中间层索引

- 修改外层索引名为 Gender

- 用下划线合并两层行索引

- 把行索引拆分为原状态

- 修改索引名为原表名称

- 恢复默认索引并将列保持为原表的相对位置



### Ex2：巧克力数据集
现有一份关于巧克力评价的数据集：

```python
df = pd.read_csv('data/chocolate.csv')

df.head(3)
Out[181]: 
    Company  Review\r\nDate Cocoa\r\nPercent Company\r\nLocation  Rating
0  A. Morin            2016              63%              France    3.75
1  A. Morin            2015              70%              France    2.75
2  A. Morin            2015              70%              France    3.00

```

1. 把列索引名中的 \n 替换为空格。

2. 巧克力 Rating 评分为1至5，每0.25分一档，请选出2.75分及以下且可可含量 Cocoa Percent 高于中位数的样本。

3. 将 Review Date 和 Company Location 设为索引后，选出 Review Date 在2012年之后且 Company Location 不属于 France, Canada, Amsterdam, Belgium 的样本。